In [1]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
from pandas import read_excel
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
from numpy import array
from statsmodels.graphics.tsaplots import plot_pacf
from tensorflow import keras

In [ ]:


# Convert time series into supervised learning problem
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # Forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # Put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# Create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return array(diff)

def prepare_data(series, n_test, n_lag, n_seq):
    # Extract raw values
    raw_values = series.values
    # Difference data to make it stationary
    diff_series = difference(raw_values, 1)
    # Rescale values to -1, 1
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_values = scaler.fit_transform(diff_series.reshape(-1, 1))
    print(max(diff_series))
    print(min(diff_series))
    # Transform into supervised learning problem X, y
    supervised = series_to_supervised(scaled_values, n_lag, n_seq)
    supervised_values = supervised.values
    # Split into train and test sets
    train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
    # Inverse transform test set to get original values
    test_values = scaler.inverse_transform(test[:, :])[:, 0]  # Extract actual values

    return scaler, train, test, test_values  # Return all needed data




def fit_lstm(train, n_lag, n_seq, n_batch, nb_epoch, n_neurons):
    # Assume train, n_lag, nb_epoch, n_batch are defined elsewhere
    X, y = train[:, 0:n_lag], train[:, n_lag:]
    X = X.reshape(X.shape[0], n_lag, 1)  # Reshape X for LSTM
    # design network
    model = Sequential()
    model.add(LSTM(n_neurons, input_shape=(X.shape[1], X.shape[2]),unroll=True)) # Remove batch_input_shape
    model.add(Dense(y.shape[1],'tanh'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    # fit network
    model.fit(X, y, epochs=nb_epoch, batch_size=n_batch, verbose=0)
    return model
# Make one forecast with an LSTM
def forecast_lstm(model, X, n_batch):
    # Reshape input pattern to [samples, timesteps, features]
    X = X.reshape(1, len(X), 1)  # Ensure there's one feature per timestep
    # Make forecast
    forecast = model.predict(X, batch_size=n_batch)
    # Convert to array
    return [x for x in forecast[0, :]]

# Evaluate the persistence model
def make_forecasts(model, n_batch, train, test, n_lag, n_seq):
    forecasts = list()
    for i in range(len(test)):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
        # Make forecast
        forecast = forecast_lstm(model, X, n_batch)
        # Store the forecast
        forecasts.append(forecast)
    return forecasts

# Invert differenced forecast
def inverse_difference(last_ob, forecast):
    # Invert first forecast
    inverted = list()
    inverted.append(forecast[0] + last_ob)
    # Propagate difference forecast using inverted first value
    for i in range(1, len(forecast)):
        inverted.append(forecast[i] + inverted[i-1])
    return inverted

# Inverse data transform on forecasts
def inverse_transform(series, forecasts, scaler, n_test):
    inverted = list()
    for i in range(len(forecasts)):
        # Create array from forecast
        forecast = array(forecasts[i])
        forecast = forecast.reshape(1, len(forecast))
        # Invert scaling
        inv_scale = scaler.inverse_transform(forecast)
        inv_scale = inv_scale[0, :]
        # Invert differencing
        index = len(series) - n_test + i - 1
        last_ob = series.values[index]
        inv_diff = inverse_difference(last_ob, inv_scale)
        # Store
        inverted.append(inv_diff)
    return inverted

# Evaluate the RMSE for each forecast time step
def evaluate_forecasts(test, forecasts, n_lag, n_seq):
    for i in range(n_seq):
        actual = [row[i] for row in test]
        predicted = [forecast[i] for forecast in forecasts]
        rmse = sqrt(mean_squared_error(actual, predicted))
        print('t+%d RMSE: %f' % ((i+1), rmse))

# Plot the forecasts in the context of the original dataset
def plot_forecasts(series, forecasts, n_test):
    # Plot the entire dataset in blue
    pyplot.plot(series.values)
    # Plot the forecasts in red
    for i in range(len(forecasts)):
        off_s = len(series) - n_test + i - 1
        off_e = off_s + len(forecasts[i]) + 1
        xaxis = [x for x in range(off_s, off_e)]
        yaxis = [series.values[off_s]] + forecasts[i]
        pyplot.plot(xaxis, yaxis, color='red')
    # Show the plot
    pyplot.xlim(left=2000)
    pyplot.show()


# Load dataset
csv_path = 'TorqueData_noFriction_NL_NG.xlsx'
series = pd.read_excel(csv_path)
series = series['Torque']
#series = combinedSeries['torque']
# Configure
n_lag = 5
n_seq = 1
n_test = round(0.3 * len(series))  # Or whatever number of tests you want
n_epochs = 20  # Number of epochs for training
n_batch = 1  # Batch size
n_neurons = 2  # Number of neurons in the LSTM layer
pyplot.plot(series.values)
# Prepare data
scaler, train, test,test_values = prepare_data(series, n_test, n_lag, n_seq)


In [ ]:
# fit model
model = fit_lstm(train, n_lag, n_seq, n_batch, n_epochs, n_neurons)
model.save('LSTMMM2.keras') # Add .keras extension to the filename
# make forecasts
forecasts = make_forecasts(model, n_batch, train, test, n_lag, n_seq)
forecasts_diff=forecasts
# inverse transform forecasts and test

forecasts = inverse_transform(series, forecasts, scaler, n_test)
actual = [row[n_lag:] for row in test]
actual = inverse_transform(series, actual, scaler, n_test)
# evaluate forecasts
evaluate_forecasts(actual, forecasts, n_lag, n_seq)
# plot forecasts
plot_forecasts(series, forecasts, n_test)

In [ ]:
import bluetooth
import serial
import glob
import sys
import time
import struct
baud = 921600
dataNumBytes = 1
CONNECTED = False
POST_PROCESSING_DONE = False
PREDICTING = True
s = serial.Serial(baudrate=baud,timeout=3, write_timeout=3,bytesize=8)
def get_serial_ports():
    """
    Lists serial ports.
    :return: ([str]) A list of available serial ports
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        #ports = glob.glob('/dev/tty[A-Za-z]*')
        #ports = glob.glob('/dev/[A-Za-z]*')
        ports = glob.glob('/dev/rfcomm*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    results = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            results.append(port)
        except (OSError, serial.SerialException):
            pass
    print("{}\n".format(results))
    return results

def connect():
    print("Attempting connection... Please wait.")
    serial_ports = get_serial_ports() 
    for serial_port in serial_ports:
        s.port = serial_port
        print("Handshaking at port: {}".format(serial_port))
        try:
            s.open()
            s.write(b'k')
            time.sleep(1)
            print("Bytes inWaiting: {}".format(s.in_waiting))
            handshake_return = s.read(1)
            print("Acknowledgement recieved: {}".format(handshake_return))               
            if handshake_return == b'M':
                print("Motor device found at port: {}".format(serial_port))
                CONNECTED = True
                print("Connected = {}".format(CONNECTED))
                s.reset_input_buffer()
                break
            else:
                s.close()
                print("no device")    
        except:
            if s.is_open:
                s.close()
            continue
def get_sample():        
        data_buffer = list()
        data = s.read(8)
        if len(data) == 8:
            for j in range(8):
                value = struct.unpack('f', data[j])
                data_buffer.append(value)
        else:
            data_buffer = [0.0,0.0]
        return data_buffer

connect()


In [4]:
def receive_float_from_esp32():
    # Read line from serial and decode it to a float
    line = s.readline().decode('utf-8').strip()
    if line:
        try:
            received_value = float(line)
            print(f"Received float from ESP32 via Bluetooth: {received_value}")
            return received_value
        except ValueError:
            print("Error: Could not convert received data to float.")
    return None

In [2]:
n_test = 5
def prepare_dataNew(series, n_test, n_lag, n_seq):
    # Extract raw values
    raw_values = series
    # Difference data to make it stationary
    diff_series = difference(raw_values, 1)
    # Rescale values to -1, 1
    #scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_values = scaler.fit_transform(diff_series.reshape(-1, 1))
    print(max(diff_series))
    print(min(diff_series))
    # Transform into supervised learning problem X, y
    # Split into train and test sets
    test = scaled_values

    return scaler,test  # Return all needed data
def forecast_lstm(model, X, n_batch):
    # Reshape input pattern to [samples, timesteps, features]
    #  # Ensure there's one feature per timestep
    X = X.reshape(1, len(X), 1) 
    # Make forecast
    forecast = model.predict(X, batch_size=n_batch)
    # Convert to array
    return [x for x in forecast[0, :]]

# Evaluate the persistence model
def make_forecasts(model, n_batch, train, test, n_lag, n_seq):
    forecasts = list()
    for i in range(len(test)):
        X, y = test[i, 0:n_lag], test[i, n_lag:]
        # Make forecast
        forecast = forecast_lstm(model, X, n_batch)
        # Store the forecast
        forecasts.append(forecast)
    return forecasts
def inverse_transform(series, forecasts, scaler, n_test):
    inverted = list()
    for i in range(len(forecasts)):
        # Create array from forecast
        forecast = array(forecasts[i])
        forecast = forecast.reshape(1, len(forecast))
        # Invert scaling
        inv_scale = scaler.inverse_transform(forecast)
        inv_scale = inv_scale[0, :]
        # Invert differencing
        index = len(series) - n_test + i - 1
        last_ob = series[index]
        inv_diff = inverse_difference(last_ob, inv_scale)
        # Store
        inverted.append(inv_diff)
    return inverted
def plot_forecasts(series, forecasts, n_test):
    # Plot the entire dataset in blue
    pyplot.plot(series)
    # Plot the forecasts in red
    for i in range(len(forecasts)):
        off_s = len(series) - n_test + i - 1
        off_e = off_s + len(forecasts[i]) + 1
        xaxis = [x for x in range(off_s, off_e)]
        yaxis = [series[off_s]] + forecasts[i]
        pyplot.plot(xaxis, yaxis, color='red')
    # Show the plot
    pyplot.show()
def readBuffer():
    try:
        # Receive the 8-byte buffer
        decoded = list()
        data = s.read(12)
        print(f"Received data: {data}")
        for i in range(3):
            Decode = data[i*4 : i*4+4]
            value, = struct.unpack('f', Decode)            
            decoded.append(value)
        # Unpack the 8 bytes into 2 floats
        # 'f' denotes a 4-byte float, and '>' denotes big-endian byte order
        #float_values = struct.unpack('>' + 'f' * 2, data)
        print("Decoded floats:", decoded)
        
        return decoded
    
    except Exception as e:
        print(f"Error: {e}")
    

In [ ]:
startT = time.time()
s.write(b'S')
Torques1 = [0.0,0.0,0.0,0.0,0.0,0.0]
Torques2 = [0.0,0.0,0.0,0.0,0.0,0.0]
TorquePre = [0.0,0.0]
v = 0
actualTau =list()
predictedTau = list()
angle = list()
interactionTorque = list()
while startT + 60 > time.time():
# Extract raw values
    pre1 = TorquePre[0]
    pre2 = TorquePre[1]
    s.write(b'G')
    s.write(b',')
    s.write(b'{pre1}')
    s.write(b',')
    s.write(b'{pre2}')
    s.write(b',')
    s.write(b'T')
    sample = readBuffer()
    torque_data1 = sample[0]
    angle_data = sample[1]
    interactionTorque_data = sample[2]
    Torques1[0] = Torques1[1]
    Torques1[1] = Torques1[2]
    Torques1[2] = Torques1[3]
    Torques1[3] = Torques1[4]
    Torques1[4] = Torques1[5]
    Torques2[0] = Torques2[1]
    Torques2[1] = Torques2[2]
    Torques2[2] = Torques2[3]
    Torques2[3] = Torques2[4]
    Torques2[4] = Torques2[5]
    Torques1[5] = torque_data1
    Torques2[5] = torque_data1
    scaler, test = prepare_dataNew(Torques1,n_test,n_lag,n_seq)
    print(test)
    X = test.reshape((1,5,1))
    torquePrediction = model.predict(X)
    forecast = inverse_transform(Torques1, torquePrediction, scaler, n_test)
    TorquePre = [forecast[0][1], forecast[0][1]]
    print("printing forecast")
    print(forecast)

    actualTau.append(torque_data1)
    predictedTau.append(forecast[0][1])
    angle.append(angle_data)
    interactionTorque.append(interactionTorque_data)

In [ ]:
for i in range(len(angle)):
    angle[i] = angle[i]/360
for j in range(len(interactionTorque)):
    if interactionTorque[j] < -50:
        interactionTorque[j] = interactionTorque[j]+114.21
    else:
        interactionTorque[j] = interactionTorque[j]

pyplot.plot(actualTau)
pyplot.plot(interactionTorque)
pyplot.plot(angle)
pyplot.show()

In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Assuming you have your torque and angle data in a DataFrame named 'data'
# with columns 'torque' and 'angle'
newTorque = pd.read_excel('C:/Users/malth/Documents/GitHub/Transparent-control-with-LSTM-and-FMG/LSTM and RNN examples/torques.xlsx')
newAngle = 360*pd.read_excel('C:/Users/malth/Documents/GitHub/Transparent-control-with-LSTM-and-FMG/LSTM and RNN examples/angles.xlsx')
newAngle = newAngle.T
newAngle.dropna(inplace=True)
newTorque = newTorque.T
newTorque.dropna(inplace=True)
combinedSeries = concat([newTorque,newAngle], axis=1)
combinedSeries.columns = ['torque','angle']
combinedSeries
data = combinedSeries

In [ ]:

# Create a new DataFrame with the desired lagged values
# Normalize torque
from sklearn.preprocessing import MinMaxScaler
torque_scaler = MinMaxScaler()
data['torque'] = torque_scaler.fit_transform(data['torque'].values.reshape(-1, 1))

# Normalize angle
angle_scaler = MinMaxScaler()
data['angle'] = angle_scaler.fit_transform(data['angle'].values.reshape(-1, 1))

data['torque_lag_1'] = data['torque'].shift(1)
data['torque_lag_2'] = data['torque'].shift(2)
data['torque_lag_3'] = data['torque'].shift(3)
data['torque_lag_4'] = data['torque'].shift(4)
data['torque_lag_5'] = data['torque'].shift(5)
data['angle_lag_1'] = data['angle'].shift(1)
data['angle_lag_2'] = data['angle'].shift(2)
data['angle_lag_3'] = data['angle'].shift(3)
data['angle_lag_4'] = data['angle'].shift(4)
data['angle_lag_5'] = data['angle'].shift(5)

# Drop rows with missing values
data.dropna(inplace=True)

# Split data into features (X) and target (y)
#X = data[['torque_lag_1', 'torque_lag_2', 'torque_lag_3', 'torque_lag_4', 'torque_lag_5',
#         'angle_lag_1', 'angle_lag_2', 'angle_lag_3', 'angle_lag_4', 'angle_lag_5']]
#X = data[['torque_lag_5', 'torque_lag_4', 'torque_lag_3', 'torque_lag_2', 'torque_lag_1',
#         'angle_lag_5', 'angle_lag_4', 'angle_lag_3', 'angle_lag_2', 'angle_lag_1']]
X = data[['torque_lag_5','angle_lag_5', 'torque_lag_4','angle_lag_4', 'torque_lag_3','angle_lag_3', 'torque_lag_2','angle_lag_2', 'torque_lag_1','angle_lag_1']]
y = data['torque']

# Normalize data (optional but often beneficial)

#scaler = MinMaxScaler()
#X_scaled = scaler.fit_transform(X)
#y_scaled = scaler.fit_transform(y.values.reshape(-1, 1))

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False,stratify=None)
print(X_train.shape[1])
# Reshape data for LSTM input
#X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
#X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_train = X_train.values.reshape(X_train.shape[0], 5, 2)  # Reshape to (samples, timesteps, features)
X_test = X_test.values.reshape(X_test.shape[0], 5, 2)
# Create LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(5,2)))
model.add(LSTM(units=50))
model.add(Dense(units=64))
model.add(Dropout(0.2))
model.add(Dense(units=1))
#model.add(LSTM(2, input_shape=(5, 2),unroll=True)) # Remove batch_input_shape
#model.add(Dense(y_train.shape[1],'tanh'))
#model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=1)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print("Test loss:", loss)

# Make predictions
predictions = model.predict(X_test)
# Invert normalization to get actual torque values
predictions_inv = torque_scaler.inverse_transform(predictions)


In [ ]:
print(X_train.shape())

In [ ]:
import matplotlib.pyplot as plt
y_actual = y_test.values
# Create a figure and axes
plt.figure(figsize=(12, 6))
plt.plot(y_actual, label='Actual')
plt.plot(predictions_inv, label='Predicted')
plt.title('Actual vs. Predicted Torque')
plt.xlabel('Time Step')
plt.ylabel('Torque')
plt.legend()
plt.show()

In [ ]:
startT = time.time()
Torques1 = [0.0,0.0,0.0,0.0,0.0]
Torques2 = [0.0,0.0,0.0,0.0,0.0]
angles1 = [0.0,0.0,0.0,0.0,0.0]
TorquePre = 0.0
s.write(b'S')
actualTau =list()
predictedTau = list()
angle = list()
interactionTorque = list()
while startT + 60 > time.time():
# Extract raw values
    pre1 = TorquePre
    pre2 = TorquePre
    s.write(b'G')
    s.write(b',')
    s.write(b'{pre1}')
    s.write(b',')
    s.write(b'{pre2}')
    s.write(b',')
    s.write(b'T')
    sample = readBuffer()
    torque_data1 = sample[0]
    save_t = torque_data1
    torque_data1 = np.array(torque_data1)
    torque_data1 = torque_data1.reshape(-1,1)
    torque_dataScaled = torque_scaler.transform(torque_data1)
    angle_data = sample[1]
    save_a = angle_data
    angle_data = np.array(angle_data)
    angle_data = angle_data.reshape(-1,1)
    angle_dataScaled = angle_scaler.transform(angle_data)
    interactionTorque_data = sample[2]
    angles1[0] = angles1[1]
    angles1[1] = angles1[2]
    angles1[2] = angles1[3]
    angles1[3] = angles1[4]
    angles1[4] = angle_dataScaled[0][0]
    Torques1[0] = Torques1[1]
    Torques1[1] = Torques1[2]
    Torques1[2] = Torques1[3]
    Torques1[3] = Torques1[4]
    Torques1[4] = torque_dataScaled[0][0]
    Torques2[0] = Torques2[1]
    Torques2[1] = Torques2[2]
    Torques2[2] = Torques2[3]
    Torques2[3] = Torques2[4]
    Torques2[4] = torque_dataScaled[0][0]
    X = [Torques1[0],angles1[0],Torques1[1],angles1[1],Torques1[2],angles1[2],Torques1[3],angles1[3],Torques1[4],angles1[4]]
    X = np.array(X)
    X = X.reshape(1, 5, 2)
    torquePrediction = model.predict(X)
    forecast = torque_scaler.inverse_transform(torquePrediction)
    TorquePre = forecast[0]
    print("printing forecast")
    print(forecast)

    actualTau.append(save_t)
    predictedTau.append(TorquePre)
    angle.append(save_a)
    interactionTorque.append(interactionTorque_data)

In [ ]:
pyplot.plot(actualTau)
pyplot.plot(predictedTau)
#pyplot.plot(angle)
pyplot.show()